# Macros 
*Nov 14, 2022*

a feature of scheme that allows you to define new special forms.


A macro is an operation performed on the source code of a program before evaluation

Macros exist in many languages, but are easiest to define correctly in a language like lisp, since the code is data (code is lists).

Scheme has a `define-macro` special form that defines a source code transformation.

**Example:**
```python
(define-macro (twice expr)
    (list 'begin expr expr)
)

# (begin (print 2) (print 2))
scm>> (twice (print 2))
2
2
```

Evaluation procedure of a macro call expression:
1. Evaluate the operator sub-expression, which evaluates to a macro
2. Call the macro procedure on the operand expression *without evaluating them first*
3. Evaluate the expression returned from the macro procedure

In short, macros take in expressions and return expression, instead of taking in values and returning values. Macros don't evaluate the operand

**Example above but without macro:**
```py
(define (twice expr)
    (list 'begin expr expr)
)

scm>> (twice '(print 2))
(begin (print 2) (print 2))

scm>> (eval (twice '(print 2)))
2
2
```

 -> Notice we need to quote the operand so that it doesn't get evaluated, and we need to call eval to evaluate the returned expression. With `define-macro` all of this is done automatically. This gives us complete control of how and when the operands are evaluated inside a macro.

**Example: printing out the operand**
```py
(define-macro (check expr)
    (list 'if expr ''passed ''failed)
)
scm>> (check (< 1 0))
failed
```
**we can also print out the failed expressions:**

```python
(define-macro (check expr)
    (list 'if expr ''passed (list 'quote (list 'failed: expr)))
)

scm>> (check (< 1 0))
(failed: (< 1 0))
```

To debug, use the regular define and pass in a quoted expression to see what the returned value is.

#### For Macro
Define a macro that evaluates an expression for each value in a sequence
```python
scm>> (for x '(2 3 4 5) (* x x))
(4 9 16 26)
```

**Solution:**
```python
(define (map fn vals)
    (if (null? vals) nil
        (cons (fn (car vals)) (map fn (cdr vals)))
    )
)
scm>> (map (lambda (x) (* x x)) '(2 3 4 5))
(4 9 16 25)


# build the expression above
(define-macro (for sym vals expr)
    (list 'map (list 'lambda (list sym) expr) vals)
)
```

### Trace Recursive Calls

In [9]:
def trace(fn):
    def traced(n):
        print(f'{fn.__name__}({n})')
        return fn(n)
    return traced

@trace
def fact(n):
    if n == 0:
        return 1
    else:
        return n * fact(n - 1)
fact(4)


fact(4)
fact(3)
fact(2)
fact(1)
fact(0)


24

```python
(define (fact n)
    (if (zero? n) 1
        (* n (fact (- n 1)))
    )
)
(define original fact)
(define (fact n) 
    (print (list 'fact n))
    (original n)
)

scm>> (fact 5)
(fact 5)
(fact 4)
(fact 3)
(fact 2)
(fact 1)
(fact 0)
120
```
Note there are some issues with this (idk what they are, rewatch lecture.) Lets fix it with `define-macro`

WTF is this shite. Rewatch the lecture.
```
(define-macro (trace expr) ; (trace (fact 5))
    (define operator (car expr)) ; fact
    `(begin
        (define original ,operator)
        (define ,operator (lambda (n)
                                (print (list (quote ,operator) n))
                                (original n)
                            )
        )
        (define result ,expr)
        (define ,operator original)
        result
    )
)
```